In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import dask.dataframe as dd
import glob

In [3]:
uni2 = pd.read_csv('UMF 85.csv')

C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,5,8,9,11,17,24,30,33,36,37,38,39,40,42,44,46,54,55,56,60,70,77,79,80,81,82,83,84,85,86,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def futureacc(x):
    if type(x)==str:
        x1 = x.split('_')
        if len(x1)>=2:
            k = x1[1]
            k1 = x1[0]
            
            if k =='WHITNEY':
                return 1
            elif k1.endswith('FU'):
                return 2
            elif k1.endswith('COLL'):
                return 3
            else:
                return 0
    else:
        return 0

In [5]:
uni2['map_marker'] = uni2['ViewData.Mapped Custodian Account'].apply(lambda x : futureacc(x))

In [6]:
uni3 = uni2[uni2['map_marker']==2.0]


In [7]:
uni3 = uni3.rename(columns = {'ViewData.Net Amount Difference':'Net Amount Difference1',
                            'ViewData.Mapped Custodian Account':'Custodian Account',
                             'ViewData.Currency':'Currency',
                             'ViewData.Ticker':'Ticker1'})

In [8]:
def subSum(numbers,total):
    length = len(numbers)
    
    
    
    if length <20:
      
            
      
        
        for index,number in enumerate(numbers):
            if np.isclose(number, total, atol=5).any():
                return [number]
                print(34567)
            subset = subSum(numbers[index+1:],total-number)
            if subset:
                #print(12345)
                return [number] + subset
        return []
    else:
        return numbers

In [9]:
def remove_mark(x,z,k):
    
   
    if ((x>1) & (x<20)):
        if ((k<6.0) & (z==0)):
            return 1
        elif ((k==0.0) & (z!=0)):
            return 1
        else:
            return 0
    else:
        return 0

In [10]:
if uni3.shape[0]!=0:
    dummy = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1'])['Net Amount Difference1'].apply(list).reset_index()
    dummy1 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1'])['ViewData.Side0_UniqueIds'].count().reset_index()
    dummy = pd.merge(dummy, dummy1 , on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1'], how = 'left')
    dummy2 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1'])['ViewData.Side1_UniqueIds'].count().reset_index()
    dummy = pd.merge(dummy, dummy2 , on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1'], how = 'left')
    dummy['sel_mark'] = dummy.apply(lambda x : 0 if ((x['ViewData.Side0_UniqueIds']==0) | (x['ViewData.Side1_UniqueIds']==0)) else 1, axis =1 )
    dummy = dummy[dummy['sel_mark']==1]
    dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))
    dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))
    dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))
    dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']
    dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))
    #dummy = pd.merge(dummy, pos , on = ['Custodian Account','Currency','Ticker1'], how = 'left')
    dummy['remove_mark'] = dummy.apply(lambda x :remove_mark(x['zero_list_len'] ,x['diff_len'], x['zero_list_sum']),axis = 1)
    dummy = dummy[['ViewData.Task Business Date','Custodian Account', 'Currency', 'Ticker1', 'zero_list',  'diff_len', 'remove_mark','sel_mark']]
    df3 = pd.merge(uni3, dummy, on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1'], how = 'left')
    df4 = df3[(df3['sel_mark']==1) & (df3['sel_mark']==1)]
    
    if df4.shape[0]!=0:
        df4['Predicted Category'] = 'Match'
        df4['Predicted Comment'] = 'Match'
        df4 = df4[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df4.to_csv('Schonfield 85 p1.csv')
        df5 = df3[~((df3['sel_mark']==1) & (df3['sel_mark']==1))]
        
        df5['Predicted Category'] = 'OTE'
        df5['Predicted Comment'] = 'Yet to be commented'
        df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df5.to_csv('Schonfield 85 p2.csv')
    else:
        df5 = df3.copy()
        
        df5['Predicted Category'] = 'OTE'
        df5['Predicted Comment'] = 'Yet to be commented'
        df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df5.to_csv('Schonfield 85 p3.csv')


C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [11]:
uni3 = uni2[uni2['map_marker']==0.0]

uni3 = uni3.rename(columns = {'ViewData.Net Amount Difference':'Net Amount Difference1',
                            'ViewData.Mapped Custodian Account':'Custodian Account',
                             'ViewData.Currency':'Currency',
                             'ViewData.Ticker':'Ticker1'})



In [12]:
if uni3.shape[0]!=0:
    dummy = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date'])['Net Amount Difference1'].apply(list).reset_index()
    dummy1 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date'])['ViewData.Side0_UniqueIds'].count().reset_index()
    dummy = pd.merge(dummy, dummy1 , on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date'], how = 'left')
    dummy2 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date'])['ViewData.Side1_UniqueIds'].count().reset_index()
    dummy = pd.merge(dummy, dummy2 , on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date'], how = 'left')
    dummy['sel_mark'] = dummy.apply(lambda x : 0 if ((x['ViewData.Side0_UniqueIds']==0) | (x['ViewData.Side1_UniqueIds']==0)) else 1, axis =1 )
    dummy = dummy[dummy['sel_mark']==1]
    dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))
    dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))
    dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))
    dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']
    dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))
    #dummy = pd.merge(dummy, pos , on = ['Custodian Account','Currency','Ticker1'], how = 'left')
    dummy['remove_mark'] = dummy.apply(lambda x :remove_mark(x['zero_list_len'] ,x['diff_len'], x['zero_list_sum']),axis = 1)
    dummy = dummy[['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date', 'zero_list',  'diff_len', 'remove_mark','sel_mark']]
    df3 = pd.merge(uni3, dummy, on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Transaction Type','ViewData.Settle Date'], how = 'left')
    df4 = df3[(df3['sel_mark']==1) & (df3['sel_mark']==1)]
    
    if df4.shape[0]!=0:
        df4['Predicted Category'] = 'Match'
        df4['Predicted Comment'] = 'Match'
        df4 = df4[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df4.to_csv('Schonfield 85 p4.csv')
        df5 = df3[~((df3['sel_mark']==1) & (df3['sel_mark']==1))]
        uni3 = df5.copy()
        dummy = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency'])['Net Amount Difference1'].apply(list).reset_index()
        dummy1 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency'])['ViewData.Side0_UniqueIds'].count().reset_index()
        dummy = pd.merge(dummy, dummy1 , on = ['ViewData.Task Business Date','Custodian Account','Currency'], how = 'left')
        dummy2 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency'])['ViewData.Side1_UniqueIds'].count().reset_index()
        dummy = pd.merge(dummy, dummy2 , on = ['ViewData.Task Business Date','Custodian Account','Currency'], how = 'left')
        dummy['sel_mark'] = dummy.apply(lambda x : 0 if ((x['ViewData.Side0_UniqueIds']==0) | (x['ViewData.Side1_UniqueIds']==0)) else 1, axis =1 )
        dummy = dummy[dummy['sel_mark']==1]
        
        dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))
        dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))
        dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))
        dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']
        dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))
        
        df3 = pd.DataFrame()
        df4 = pd.DataFrame()
        df5 = pd.DataFrame()
        
        uni3.drop(['zero_list', 'diff_len', 'remove_mark', 'sel_mark'], axis = 1, inplace = True)
        
        dummy['remove_mark'] = dummy.apply(lambda x :remove_mark(x['zero_list_len'] ,x['diff_len'], x['zero_list_sum']),axis = 1)
        dummy = dummy[['ViewData.Task Business Date','Custodian Account','Currency','zero_list',  'diff_len', 'remove_mark','sel_mark']]
        df3 = pd.merge(uni3, dummy, on = ['ViewData.Task Business Date','Custodian Account','Currency'], how = 'left')
        df4 = df3[(df3['sel_mark']==1) & (df3['sel_mark']==1)]
        
        
        if df4.shape[0]!=0:
            
            df4['Predicted Category'] = 'Match'
            df4['Predicted Comment'] = 'Match'
            df4 = df4[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
            df4.to_csv('Schonfield 85 p5.csv')
        
        
            df5 = df3[~((df3['sel_mark']==1) & (df3['sel_mark']==1))]
            df5['Predicted Category'] = 'OTE'
            df5['Predicted Comment'] = 'Yet to be commented'
            df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
            df5.to_csv('Schonfield 85 p6.csv')
        else:
            df5 = df3.copy()
        
            df5['Predicted Category'] = 'OTE'
            df5['Predicted Comment'] = 'Yet to be commented'
            df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
            df5.to_csv('Schonfield 85 p7.csv')
    else:
        df5 = df3.copy()
        
        df5['Predicted Category'] = 'OTE'
        df5['Predicted Comment'] = 'Yet to be commented'
        df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df5.to_csv('Schonfield 85 p8.csv')
        
    
    

C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [13]:
uni3 = uni2[uni2['map_marker']==1.0]

uni3 = uni3.rename(columns = {'ViewData.Net Amount Difference':'Net Amount Difference1',
                            'ViewData.Mapped Custodian Account':'Custodian Account',
                             'ViewData.Currency':'Currency',
                             'ViewData.Ticker':'Ticker1'})

In [14]:
if uni3.shape[0]!=0:
    dummy = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date'])['Net Amount Difference1'].apply(list).reset_index()
    dummy1 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date'])['ViewData.Side0_UniqueIds'].count().reset_index()
    dummy = pd.merge(dummy, dummy1 , on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date'], how = 'left')
    dummy2 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date'])['ViewData.Side1_UniqueIds'].count().reset_index()
    dummy = pd.merge(dummy, dummy2 , on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date'], how = 'left')
    dummy['sel_mark'] = dummy.apply(lambda x : 0 if ((x['ViewData.Side0_UniqueIds']==0) | (x['ViewData.Side1_UniqueIds']==0)) else 1, axis =1 )
    dummy = dummy[dummy['sel_mark']==1]
    dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))
    dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))
    dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))
    dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']
    dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))
    #dummy = pd.merge(dummy, pos , on = ['Custodian Account','Currency','Ticker1'], how = 'left')
    dummy['remove_mark'] = dummy.apply(lambda x :remove_mark(x['zero_list_len'] ,x['diff_len'], x['zero_list_sum']),axis = 1)
    dummy = dummy[['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date', 'zero_list',  'diff_len', 'remove_mark','sel_mark']]
    df3 = pd.merge(uni3, dummy, on = ['ViewData.Task Business Date','Custodian Account','Currency','Ticker1','ViewData.Settle Date'], how = 'left')
    df4 = df3[(df3['sel_mark']==1) & (df3['sel_mark']==1)]
    
    if df4.shape[0]!=0:
        df4['Predicted Category'] = 'Match'
        df4['Predicted Comment'] = 'Match'
        df4 = df4[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df4.to_csv('Schonfield 85 p9.csv')
        df5 = df3[~((df3['sel_mark']==1) & (df3['sel_mark']==1))]
        uni3 = df5.copy()
        dummy = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency'])['Net Amount Difference1'].apply(list).reset_index()
        dummy1 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency'])['ViewData.Side0_UniqueIds'].count().reset_index()
        dummy = pd.merge(dummy, dummy1 , on = ['ViewData.Task Business Date','Custodian Account','Currency'], how = 'left')
        dummy2 = uni3.groupby(['ViewData.Task Business Date','Custodian Account','Currency'])['ViewData.Side1_UniqueIds'].count().reset_index()
        dummy = pd.merge(dummy, dummy2 , on = ['ViewData.Task Business Date','Custodian Account','Currency'], how = 'left')
        dummy['sel_mark'] = dummy.apply(lambda x : 0 if ((x['ViewData.Side0_UniqueIds']==0) | (x['ViewData.Side1_UniqueIds']==0)) else 1, axis =1 )
        dummy = dummy[dummy['sel_mark']==1]
        
        dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))
        dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))
        dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))
        dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']
        dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))
        
        df3 = pd.DataFrame()
        df4 = pd.DataFrame()
        df5 = pd.DataFrame()
        
        uni3.drop(['zero_list', 'diff_len', 'remove_mark', 'sel_mark'], axis = 1, inplace = True)
        
        dummy['remove_mark'] = dummy.apply(lambda x :remove_mark(x['zero_list_len'] ,x['diff_len'], x['zero_list_sum']),axis = 1)
        dummy = dummy[['ViewData.Task Business Date','Custodian Account','Currency','zero_list',  'diff_len', 'remove_mark','sel_mark']]
        df3 = pd.merge(uni3, dummy, on = ['ViewData.Task Business Date','Custodian Account','Currency'], how = 'left')
        df4 = df3[(df3['sel_mark']==1) & (df3['sel_mark']==1)]
        
        
        if df4.shape[0]!=0:
            
            df4['Predicted Category'] = 'Match'
            df4['Predicted Comment'] = 'Match'
            df4 = df4[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
            df4.to_csv('Schonfield 85 p10.csv')
        
        
            df5 = df3[~((df3['sel_mark']==1) & (df3['sel_mark']==1))]
            df5['Predicted Category'] = 'OTE'
            df5['Predicted Comment'] = 'Yet to be commented'
            df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
            df5.to_csv('Schonfield 85 p11.csv')
        else:
            df5 = df3.copy()
        
            df5['Predicted Category'] = 'OTE'
            df5['Predicted Comment'] = 'Yet to be commented'
            df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
            df5.to_csv('Schonfield 85 p12.csv')
    else:
        df5 = df3.copy()
        
        df5['Predicted Category'] = 'OTE'
        df5['Predicted Comment'] = 'Yet to be commented'
        df5 = df5[['ViewData.Side0_UniqueIds','ViewData.Side1_UniqueIds','Predicted Category','Predicted Comment']]
        df5.to_csv('Schonfield 85 p13.csv')
        
    
    

C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_